# Importing Python Notebooks

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statistics import mean,mode,median 
import csv
from tqdm import tqdm
import time
%matplotlib inline

# Loading Data

In [ ]:
dataset=pd.read_csv("all_data_19_07_19.tsv",delimiter="\t")
dataset.head()

# Data Wrangling and Cleaning

## Trim 1: Extract Relevant Columns

In [ ]:
df = dataset[['ENTITYA','TYPEA','ENTITYB','TYPEB','MECHANISM','EFFECT','DIRECT']]
df.head()

In [ ]:
print(df.info())
print('-------------------------------')
print(df['TYPEA'].value_counts())
print('-------------------------------')
print(df['TYPEB'].value_counts())

We further trim our dataset by limiting our entities to either proteins, complexes, or protein families.

In [ ]:
relevant_types = ['protein','complex','proteinfamily']
df = df[df['TYPEA'].isin(relevant_types)]
df = df[df['TYPEB'].isin(relevant_types)]

In [ ]:
print(df.info())
print('-------------------------------')
print(df['TYPEA'].value_counts())
print('-------------------------------')
print(df['TYPEB'].value_counts())

We further trim our dataset by limiting our connections to direct connections

In [ ]:
df = df[df['DIRECT'] == 'YES']
print(df.info())
print('-------------------------------')
print(df['TYPEA'].value_counts())
print('-------------------------------')
print(df['TYPEB'].value_counts())

Lastly, we remove vagueness from EFFECT by removing the all interactions with an unknown effect.

In [ ]:
df = df[df['EFFECT'] != 'unknown']
print(df.info())
print('-------------------------------')
print(df['EFFECT'].value_counts())

## Simplifying Column Values

For the purpose of this study, we will be simplifying our effects column by generalizing the different types to either up-regulating, down-regulating, or complex-forming.

In [ ]:
df['EFFECT'].replace(regex={r'(up-regulates).*$': 'up', r'(down-regulates).*$': 'down','form complex':'complex'},inplace = True)
df['EFFECT'].value_counts()

# Loading Graph via NetworkX

In [ ]:
df_graph = df[['ENTITYA','ENTITYB','MECHANISM','EFFECT']]
Graph = nx.from_pandas_edgelist(df_graph,'ENTITYA','ENTITYB',edge_attr = ['MECHANISM','EFFECT'], create_using = nx.Graph())
G_mech = nx.get_edge_attributes(Graph,'MECHANISM')
G_effect = nx.get_edge_attributes(Graph,'EFFECT')

## Network Trimming with respect to effect

We trim and retain edges within a network with respect to the following condition statements:

* The **succeeding/preceeding node** must have the **same type of regulation** as the node of interest, else trim it off **unless** \
* The connection is a **binding mechanism** \
* The **succeeding/preceeding node** is a **complex** \
* **Parameters are easily configured**

To initialize our trimming function, we make an initial seed with no nodes and edges that we will grow our trimmed network from.

In [ ]:
def build_sappling(seed):
    G = nx.Graph()
    states = {seed:['root']}
    edges = set(Graph.edges(seed))
    G.add_edges_from(Graph.edges(seed))
    for edge in edges:
        try:
            state = G_effect[edge]
            states[edge[0]] = [state]
        except KeyError:
            edge_inv = tuple([edge[1], edge[0]])
            state = G_effect[edge_inv]
            states[edge[1]] = [state]
            
    return G,states

In [ ]:
sappling,states = build_sappling('FOXM1')
sappling.edges()

In [ ]:
def remove_copies(branches):
    unique_branches = []
    for branch in branches:
        branch1 = branch
        branch2 = (branch[1],branch[0])
        if (branch1 or branch2) not in set (sappling.edges):
            unique_branches.append(branch1)
    return unique_branches


def grow_tree(root,tree_branches,states):
    orig_edges = set(root.edges())
    time.sleep(1.0)
    for branch in tqdm(set(tree_branches)):
        bud = branch[1]
        try:
            bud_state = states[bud]
        except KeyError:
            continue
        potential_branches = remove_copies(set(Graph.edges(bud)))
        for pb in potential_branches:
            try:
                state = G_effect[pb]
                mech = G_mech[pb]
                node = pb[1]
            except KeyError:
                pb_inv = (pb[1],pb[0])
                state = G_effect[pb_inv]
                mech = G_mech[pb_inv]
                node = pb[0]
            #print(node)
            if ((state in bud_state) or ('complex' in bud_state) or ('complex' in state) or (mech == 'binding')):
                root.add_edge(pb[0],pb[1])
                if node not in states.keys():
                    states[node] = [state]
                elif node in states.keys():
                    states_vals = set(states[node]).union(set([state]))
                    states[node] = list(states_vals)
    updated_edges = set(root.edges())
    edge_diff = len(updated_edges)-len(orig_edges)
    new_edges = updated_edges.difference(orig_edges)
    time.sleep(1.0)
    print('# of new edges:',edge_diff)
    #print(new_edges)
    return edge_diff,new_edges


In [ ]:
edge_diff = 1
lvl = 0
while edge_diff != 0:
    print('-------------')
    print(f'@ Level {lvl}:')
    if lvl == 0:
        edge_diff,new_edges = grow_tree(sappling,sappling.edges(),states)
    else:
        edge_diff,new_edges = grow_tree(sappling,new_edges,states)
    lvl +=1

In [ ]:
print('Original Network Logistics:')
print('----------------------------------------')
print(f'Total Nodes:{len(Graph.nodes())}')
print(f'Total Edges:{len(Graph.edges())}')
print('----------------------------------------')
print('Trimmed Network Logistics:')
print('----------------------------------------')
print(f'Total Nodes:{len(sappling.nodes())}')
print(f'Total Edges:{len(sappling.edges())}')
print('----------------------------------------')

# Implementing Minimum Leaf Spanning Tree Algorithm

## Undirected Implementation

In [ ]:
Tree = sappling.copy()

In [ ]:
V = set(Tree.nodes())
v = 'FOXM1'
MCDS = set([v])
edges = set(Tree.edges(v))
W = set()
for x in edges:
    W.add(x[1])
U = set([v]) | W
while U != V:
    w = None
    w_length = 0
    for node in W:
        nghbr = set(Tree.edges(node))
        Gnode = set()
        for x in nghbr:
            Gnode.add(x[1])
        nghbrs = set(Gnode) - U
        if len(nghbrs) > w_length:
            w_length = len(nghbrs)
            w = node
    MCDS = MCDS | set([w])
    print(MCDS)
    neighborhood = set(Tree.edges(w))
    edg = set()
    for x in neighborhood:
        edg.add(x[1])
    U = U | edg
    W = (W - set([w])) | (edg - set([v]))
    v = w

In [ ]:
len(MCDS)

In [ ]:
with open('FOXM1_path.txt', 'w') as filehandle:
    for x in MCDS:
        filehandle.writelines("%s, " % x)

# Directed Implementation

In [ ]:
directedTree = sappling.to_directed()

## Version 1 : Treat Directed Graph as Undirected

Note that this version is simply an implementation of the undirected MCDS algorithm and simply configures it for use in the case of a directed graph.

In [ ]:
# INITIALIZATION OF CODE SNIPET
V = set(directedTree.nodes()) #Sets V as all nodes
# CHOOSE A NODE V WITH HIGHEST DEGREE
v = 'FOXM1'
MCDS = set([v]) #Outputs Node with most connections
inward = set(directedTree.in_edges(v))
outward = set(directedTree.out_edges(v))
W = set()
for x in inward:
    W.add(x[0])
for y in outward:
    W.add(y[1])
#W = set(directedTree[v]) #Saves Nodes which are connected to node with most connections
U = set([v]) | W
while U != V:
    w = None
    w_length = 0
    for node in W:
        inward2 = set(directedTree.in_edges(node))
        outward2 = set(directedTree.out_edges(node))
        Gnode = set()
        for x in inward2:
              Gnode.add(x[0])
        for y in outward2:
              Gnode.add(y[1])
        neighbours = set(Gnode) - U
        if (len(neighbours) > w_length):
            w_length = len(neighbours)
            w = node
    MCDS = MCDS | set([w])
    print(MCDS)
    inward3 = set(directedTree.in_edges(w))
    outward3 = set(directedTree.out_edges(w))
    edi = set()
    for x in inward3:
        edi.add(x[0])
    for y in outward3:
        edi.add(y[1])
    U = U | edi
    W = (W - set([w])) | (edi - set([v]))
    v = w

In [ ]:
len(MCDS)

In [ ]:
with open('FOXM1_path_directedv1.txt', 'w') as filehandle:
    for x in MCDS:
        filehandle.writelines("%s, " % x)

## Version 2: Directed Graph Implementation inspired from MLST